In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import matplotlib.pyplot as plt

thispath = '/Users/zhangchi/富国实习/组合优化/组合优化数据1118'

In [2]:
mu = pd.read_csv(thispath+'/最终结果/2018-01-31.csv', index_col=[0])
mu = 0.01*mu
mu.index.values.tolist().index('000157.SZ')

1

In [3]:
date = parse(mu.columns[0])
date

datetime.datetime(2018, 1, 31, 0, 0)

In [77]:
date = datetime.strftime(date, '%Y-%m-%d')
date
type(mu.columns[0])

str

In [4]:
date_idx = [date]*3533
# for i in range(3533):
#     date_idx.append(date)
mu.index = [pd.to_datetime(date_idx), mu.index]
mu

2018/01/31  0:00:00
2018-01-31 600177.SH             0.010661
           000157.SZ             0.010624
           002415.SZ             0.010546
           600887.SH             0.010486
           002191.SZ             0.010441
...                                   ...
           601360.SH            -0.023308
           300708.SZ            -0.023550
           300487.SZ            -0.023627
           002908.SZ            -0.023777
           002856.SZ            -0.026124

[3533 rows x 1 columns]

In [6]:
weight = pd.read_csv(thispath+'/HS300成分权重.csv', index_col=[0])
weight.index = pd.to_datetime(weight.index.values)
weight

,代码,权重
2010-01-29,600246.SH,0.000900
2010-01-29,600251.SH,0.000900
2010-01-29,600256.SH,0.002399
2010-01-29,600266.SH,0.001099
2010-01-29,600269.SH,0.001899
...,...,...
2019-10-31,601577.SH,0.000263
2019-10-31,601162.SH,0.000251
2019-10-31,601212.SH,0.000242
2019-10-31,600299.SH,0.000237


In [7]:
index2 = weight['代码']
weight.drop('代码', axis=1, inplace=True)
weight.index = [weight.index, index2]
weight

权重
           代码                 
2010-01-29 600246.SH  0.000900
           600251.SH  0.000900
           600256.SH  0.002399
           600266.SH  0.001099
           600269.SH  0.001899
...                        ...
2019-10-31 601577.SH  0.000263
           601162.SH  0.000251
           601212.SH  0.000242
           600299.SH  0.000237
           600733.SH  0.000184

[35400 rows x 1 columns]

In [8]:
from optimizer import *
stockPool = weight

# 3.市值
size = pd.read_csv(inpath + '/流通市值.csv', index_col=[0])
size = change_format(size)

# 4、中信一级行业
industry = pd.read_csv(inpath + '/中信一级行业.csv', index_col=[0], encoding='gbk')
industry = change_format(industry)

# 5、基准指数
# base_weight = pd.read_csv(inpath + '/中证500成份权重.csv', index_col=[0])
base_weight = pd.read_csv(inpath + '/沪深300成份权重.csv', index_col=[0])
base_weight = change_format(base_weight).fillna(0)

stock_data = pd.concat([stockPool, mu, size, industry, base_weight], join_axes=[stockPool.index], axis=1)
stock_data.columns = ['指数权重', '预期收益', '流通市值', '行业', '基准指数权重']

# 加入交易条件限制，即剔除不满足条件的股票
stock_data = filter_data(stock_data)
stock_data

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
/Users/zhangchi/PycharmProjects/portfolio_opt2/optimizer.py:53: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  data = pd.concat([data, price, state, listlen, ST, zhandie], join_axes=[data.index],axis=1)


指数权重      预期收益          流通市值     行业   基准指数权重  \
date       stcode                                                        
2018-01-31 000001.SZ  0.009701 -0.006222  2.376978e+07     银行  0.00970   
           000002.SZ  0.014681  0.007157  3.646762e+07    房地产  0.01468   
           000060.SZ  0.001610 -0.000831  2.653307e+06   有色金属  0.00161   
           000063.SZ  0.005950 -0.004101  1.068061e+07     通信  0.00595   
           000069.SZ  0.002730  0.002436  6.362424e+06    房地产  0.00273   
...                        ...       ...           ...    ...      ...   
           603160.SH  0.000310 -0.002506  1.949678e+06  电子元器件  0.00031   
           603799.SH  0.002380 -0.006211  5.856294e+06   有色金属  0.00238   
           603833.SH  0.000480 -0.003650  5.933439e+05   轻工制造  0.00048   
           603993.SH  0.001610 -0.002025  9.793020e+06   有色金属  0.00161   
           603858.SH  0.000300 -0.000256  1.617866e+06     医药  0.00030   

                            月末收盘价  state    上市天数 特殊处理  涨跌停  
date       stcode                                           
2018-01-31 000001.SZ  1493.638682    1.0  9801.0    1  0.0  
           000002.SZ  5156.990066    1.0  9865.0    1  0.0  
           000060.SZ   283.962336    1.0  7679.0    1  0.0  
           000063.SZ   491.306222    1.0  7380.0    1  0.0  
           000069.SZ   383.324711    1.0  7449.0    1  0.0  
...                           ...    ...     ...  ...  ...  
           603160.SH    85.246562    1.0   472.0    1  0.0  
           603799.SH   100.640600    1.0  1099.0    1  0.0  
           603833.SH   142.940000    1.0   310.0    1  0.0  
           603993.SH    24.402145    1.0  1941.0    1  0.0  
           603858.SH    50.527984    1.0   440.0    1  0.0  

[299 rows x 10 columns]

In [1]:
stock_data_M = stock_data.loc['2018-01']
e_ret = stock_data_M['预期收益'].values.tolist()
idtry = pd.get_dummies(stock_data_M['行业']).values.tolist()
cap = stock_data_M['市值'].values.tolist()
wb = stock_data_M['基准指数权重'].values.tolist()


NameError: name 'stock_data' is not defined

In [5]:
rand_mat = np.random.rand(22,50)
rand_mat

array([[0.5262271 , 0.53226451, 0.96882542, ..., 0.67470188, 0.73672262,
        0.22414285],
       [0.11752955, 0.59610483, 0.80457238, ..., 0.16157252, 0.20302534,
        0.72925671],
       [0.36373617, 0.07682214, 0.74407567, ..., 0.66079763, 0.28304612,
        0.50183159],
       ...,
       [0.39227704, 0.02514191, 0.82117727, ..., 0.12287901, 0.72550519,
        0.49705435],
       [0.9438384 , 0.8859287 , 0.33722542, ..., 0.72112652, 0.4161298 ,
        0.38334435],
       [0.39222205, 0.79875263, 0.42833563, ..., 0.54428299, 0.20008951,
        0.86399909]])

In [12]:
rand_cov = np.cov(rand_mat, rowvar=True)
rand_corr = np.corrcoef(rand_mat, rowvar=True)
eig_corr, eigVec_corr = np.linalg.eig(rand_corr)
eig_corr, eigVec_corr


(array([2.21754324, 2.13259871, 1.79716504, 0.13299337, 0.20176143,
        0.31898469, 0.35433118, 0.37451719, 0.47029649, 0.54072959,
        0.59005553, 0.73749404, 0.81399154, 0.92159393, 0.99418132,
        1.04119256, 1.56423941, 1.44017713, 1.46444492, 1.45221756,
        1.19446434, 1.24502679]),
 array([[-4.04760303e-01, -1.15012218e-01,  1.19705548e-01,
          1.95296966e-01,  1.56460752e-01,  3.54726172e-01,
         -2.67936113e-01, -5.66099242e-02,  1.33110709e-01,
          2.75710725e-01, -3.86614733e-02, -7.02034486e-02,
          1.73760307e-01, -4.01964563e-01,  3.95419357e-01,
          1.82321122e-01, -1.09286101e-01,  1.18048850e-01,
          3.36721043e-02,  1.08492526e-01,  5.52376016e-03,
         -1.57569824e-01],
        [-3.94284263e-02,  4.07685954e-02,  3.10254937e-01,
          4.69538279e-02,  3.96421232e-01, -3.33837819e-02,
          2.86803030e-01, -9.21533218e-02, -1.14397766e-01,
          3.31827322e-03, -2.08328899e-02,  1.48518759e-01,
       